<a href="https://colab.research.google.com/github/bhattacharjee/msc-ai-project/blob/main/DeepANN/deep_ann_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
USE_FLOAT32 = False
REDUCE_DF_SIZE = True


import tensorflow as tf
from keras.utils import np_utils
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import logging, sys, random, glob
from google.colab import drive
from functools import lru_cache
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from tensorflow import keras as K
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, losses
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Model
import IPython

from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score

import gc


drive.mount("/content/drive")

def set_random_seed():
    np.random.seed(1)
    random.seed(1)
    tf.random.set_seed(1)

root = logging.getLogger()
root.setLevel(logging.INFO)
handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
root.addHandler(handler)

!if [[ -d /content/drive/MyDrive/MSCPROJDATA && ! -d MSCPROJDATA ]]; then cp -r /content/drive/MyDrive/MSCPROJDATA .; fi

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
dataset_names = ["baseline", "advanced-only", "fourier-only", "baseline-and-fourier", "advanced-and-fourier", "advanced", "fourier"]
def get_columns_from_df(thisdf):
    baseline_columns = [c for c in thisdf.columns if c.startswith('baseline') and "head" not in c and "tail" not in c]
    baseline_columns = [c for c in baseline_columns if "filesize" not in c]
    baseline_columns = [c for c in baseline_columns if "begin" not in c and "end" not in c]

    advanced_columns = [c for c in thisdf.columns if "advanced" in c]
    advanced_columns = [c for c in advanced_columns if "begin" not in c and "end" not in c]
    advanced_columns = [c for c in advanced_columns if "head" not in c and "tail" not in c]
    advanced_columns = [c for c in advanced_columns if "start" not in c]
    advanced_columns_only = list(set(advanced_columns))
    advanced_columns = list(set(advanced_columns + baseline_columns))

    fourier_columns = [c for c in thisdf.columns if "fourier" in c and "value" not in c]
    fourier_columns = [c for c in fourier_columns if "1byte" in c]
    fourier_columns = [c for c in fourier_columns if "begin" not in c and "end" not in c]
    fourier_columns = [c for c in fourier_columns if "head" not in c and "tail" not in c]
    fourier_columns = [c for c in fourier_columns if "start" not in c]
    fourier_columns_only = list(set(fourier_columns))
    fourier_columns = list(set(advanced_columns + fourier_columns))
    
    baseline_and_advanced = list(set(baseline_columns + advanced_columns_only))
    baseline_and_fourier = list(set(baseline_columns + fourier_columns_only))
    advanced_and_fourier = list(set(advanced_columns_only + fourier_columns_only))

    all_columns = [c for c in thisdf.columns]
    
    return {\
        "baseline": baseline_columns,\
        "advanced-only": advanced_columns_only,\
        "fourier-only": fourier_columns_only,\
        "baseline-and-fourier": baseline_and_fourier,\
        "advanced-and-fourier": advanced_and_fourier,\
        "advanced": advanced_columns,\
        "fourier": fourier_columns,\
        "all_columns": all_columns,
    }


def reduce_df_size(df):
    original_usage = df.memory_usage().sum() / (1024**2)
    for col in df.columns:
        if "fourier.value" in col or \
            "tail" in col or \
            "head" in col or \
            "end" in col or \
            "begin" in col:
            df.drop([col], axis=1, inplace=True)
            continue
        if USE_FLOAT32:
            if "float64" in str(df[col].dtype):
                df[col] = df[col].astype('float32')
        elif "int64" in str(df[col].dtype):
            df[col] = df[col].astype("int8")
    new_usage = df.memory_usage().sum() / (1024**2)
    print(f"Reduced memory sage from {original_usage} to {new_usage}")
    return df

def call_gc():
    for i in range(3):
        for j in range(3):
            gc.collect(j)

@lru_cache(maxsize=5)
def load_datasets():
    datasets = dict()
    for file in glob.glob("MSCPROJDATA/**.parquet.gz", recursive=True):
        if not file.startswith("MSCPROJDATA/n1"):
            print(f"Loading {file}")
            df = pd.read_parquet(file)
            datasets[file] = df
            df["is_encrypted"] = 1 if "encr" in file.lower() else 0
    
            if REDUCE_DF_SIZE:
                df = reduce_df_size(df)

            df = df.sample(frac=1).reset_index(drop=True)
            datasets[file] = df
            call_gc()

    return datasets

@lru_cache(maxsize=5)
def get_columns():
    datasets = load_datasets()
    df = list(datasets.values())[0]
    return get_columns_from_df(df)

_ = load_datasets()
_ = get_columns()


Loading MSCPROJDATA/plaintext.expanded.parquet.gz
Reduced memory sage from 417.6138381958008 to 50.274044036865234
Loading MSCPROJDATA/expanded.pyencrypted_v1.b32.parquet.gz
Reduced memory sage from 417.6138381958008 to 47.575284004211426
Loading MSCPROJDATA/plaintext.combined.parquet.gz
Reduced memory sage from 417.6138381958008 to 50.274044036865234
Loading MSCPROJDATA/expanded.pyencrypted_v1.parquet.gz
Reduced memory sage from 417.6138381958008 to 49.19454002380371
Loading MSCPROJDATA/expanded.pyencrypted_v2.parquet.gz
Reduced memory sage from 417.6138381958008 to 48.65478801727295
Loading MSCPROJDATA/expanded.plaintext.base32.parquet.gz
Reduced memory sage from 417.6138381958008 to 48.65478801727295
Loading MSCPROJDATA/plaintext.base32.combined.parquet.gz
Reduced memory sage from 417.6138381958008 to 48.65478801727295
Loading MSCPROJDATA/expanded.pyencrypted_v2.base32.parquet.gz
Reduced memory sage from 417.6138381958008 to 48.65478801727295


In [21]:
@lru_cache(maxsize=5)
def get_train_test_df():
    datasets = load_datasets()
    df = pd.concat([v for v in datasets.values()])
    print(df.describe().T)
    x_cols = [str(c) for c in df.columns if "is_encrypted" != str(c)]
    y_cols = "is_encrypted"

    df = df.sample(frac=1).reset_index(drop=True)
    df.fillna(0.0)

    X = df[x_cols]
    y = df[y_cols]

    min_max_scaler = MinMaxScaler()
    X = min_max_scaler.fit_transform(X)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    return X_train, X_test, y_train, y_test

_ = get_train_test_df()

                                  count           mean           std  \
Unnamed: 0                     646824.0   9.378131e-01  7.390309e+01   
baseline.shannon_entropy       646824.0   6.107507e+00  1.519591e+00   
baseline.montecarlo_pi         646824.0   2.061952e+00  1.167288e+00   
baseline.chisquare_full        646824.0   8.482973e+06  5.999312e+07   
baseline.autocorrelation_full  646824.0   6.241427e-02  1.416467e-01   
...                                 ...            ...           ...   
fourier.stat.4byte.moment.12   646824.0  2.019296e+231           inf   
fourier.stat.4byte.moment.13   646824.0  9.582912e+250           inf   
fourier.stat.4byte.moment.14   646824.0  4.786819e+270           inf   
fourier.stat.4byte.moment.15   646824.0  2.523486e+290           inf   
is_encrypted                   646824.0   5.000000e-01  5.000004e-01   

                                         min            25%            50%  \
Unnamed: 0                     -1.280000e+02  -6.300000e+

In [19]:
list(load_datasets().values())[0].columns

Index(['Unnamed: 0', 'baseline.shannon_entropy', 'baseline.montecarlo_pi',
       'baseline.chisquare_full', 'baseline.autocorrelation_full',
       'baseline.filesize', 'advanced.kurtosis_full', 'advanced.skew_full',
       'advanced.moment_full.2', 'advanced.moment_full.3',
       'advanced.moment_full.4', 'advanced.moment_full.5',
       'advanced.moment_full.6', 'advanced.moment_full.7',
       'advanced.moment_full.8', 'advanced.moment_full.9',
       'advanced.moment_full.10', 'advanced.moment_full.11',
       'advanced.moment_full.12', 'advanced.moment_full.13',
       'advanced.moment_full.14', 'advanced.moment_full.15',
       'advanced.dit.renyi.full.0', 'advanced.dit.renyi.full.1',
       'advanced.dit.renyi.full.2', 'advanced.dit.renyi.full.3',
       'advanced.dit.renyi.full.4', 'advanced.dit.renyi.full.5',
       'advanced.dit.renyi.full.6', 'advanced.dit.renyi.full.7',
       'advanced.dit.renyi.full.8', 'advanced.dit.renyi.full.9',
       'advanced.dit.renyi.full.10', '

In [20]:
class Model:
    @staticmethod
    def print_and_plot(model):
        print(model.summary())
        plot = tf.keras.utils.plot_model(
            model,
            show_shapes=True,
            expand_nested=True)
        IPython.display.display(plot)

    @staticmethod
    def create_common_part():
        model = tf.keras.Sequential(
            [
                layers.Dense(4, activation = 'relu', name="Dense-4"),
                layers.Dropout(0.2),
                layers.Dense(2, activation = 'relu', name="Dense-2"),
                layers.Dense(1, activation = 'sigmoid', name="Final Layer")
            ]
        )
        return model

    @staticmethod
    def create(name:str, columns:list):
        X_train, X_test, y_train, y_test = get_train_test_df()
        X_train = X_train[columns].to_numpy()

        model = tf.keras.Sequential(
            [
                layers.Dense(8, input_dim=X_train.shape[1], activation='relu', name="Dense-8"),
                layers.Dropout(0.2),
                Model.create_common_part(),
            ]
        )


NameError: ignored

In [11]:
list(get_columns().values())[0]

['baseline.shannon_entropy',
 'baseline.montecarlo_pi',
 'baseline.chisquare_full',
 'baseline.autocorrelation_full']